In [4]:
import pandas as pd

# Load the dataset
data_path = "../WA_Fn-UseC_-HR-Employee-Attrition.csv"
df = pd.read_csv(data_path)

# Mapping dictionaries as per user description
education_map = {1: 'Below College', 2: 'College', 3: 'Bachelor', 4: 'Master', 5: 'Doctor'}
env_satisfaction_map = job_involvement_map = job_satisfaction_map = relationship_satisfaction_map = {1: 'Low', 2: 'Medium', 3: 'High', 4: 'Very High'}
performance_rating_map = {1: 'Low', 2: 'Good', 3: 'Excellent', 4: 'Outstanding'}
worklife_balance_map = {1: 'Bad', 2: 'Good', 3: 'Better', 4: 'Best'}

# Apply mappings
df['Education'] = df['Education'].map(education_map)
df['EnvironmentSatisfaction'] = df['EnvironmentSatisfaction'].map(env_satisfaction_map)
df['JobInvolvement'] = df['JobInvolvement'].map(job_involvement_map)
df['JobSatisfaction'] = df['JobSatisfaction'].map(job_satisfaction_map)
df['PerformanceRating'] = df['PerformanceRating'].map(performance_rating_map)
df['RelationshipSatisfaction'] = df['RelationshipSatisfaction'].map(relationship_satisfaction_map)
df['WorkLifeBalance'] = df['WorkLifeBalance'].map(worklife_balance_map)

# Prepare summary metrics
total_employees = len(df)
total_attrition = df['Attrition'].value_counts().get('Yes', 0)
attrition_rate = round((total_attrition / total_employees) * 100, 2)
avg_income = round(df['MonthlyIncome'].mean(), 2)
avg_distance = round(df['DistanceFromHome'].mean(), 2)
overtime_pct = round((df['OverTime'] == 'Yes').sum() / total_employees * 100, 2)
avg_wlb = df['WorkLifeBalance'].mode()[0]

# Create summary dataframe
summary_df = pd.DataFrame({
    'Metric': ['Total Employees', 'Total Attrition Cases', 'Attrition Rate (%)', 'Average Monthly Income', 
               'Average Distance From Home', '% Working Overtime', 'Most Common WorkLife Balance'],
    'Value': [total_employees, total_attrition, attrition_rate, avg_income, avg_distance, overtime_pct, avg_wlb]
})



with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    # Write summary
    summary_df.to_excel(writer, sheet_name='Summary', index=False)

    # Chart 1: Distance from Home by Job Role and Attrition
    pivot_distance = df.pivot_table(values='DistanceFromHome', index='JobRole', columns='Attrition', aggfunc='mean')
    pivot_distance.to_excel(writer, sheet_name='Charts', startrow=1, startcol=0)

    workbook  = writer.book
    worksheet = writer.sheets['Charts']

    chart1 = workbook.add_chart({'type': 'column'})
    chart1.add_series({
        'name':       'Attrition = Yes',
        'categories': ['Charts', 2, 0, 2 + len(pivot_distance)-1, 0],
        'values':     ['Charts', 2, 2, 2 + len(pivot_distance)-1, 2],
    })
    chart1.set_title({'name': 'Avg Distance from Home by Job Role & Attrition'})
    worksheet.insert_chart('E2', chart1)

    # Chart 2: Avg Monthly Income by Education & Attrition
    pivot_income = df.pivot_table(values='MonthlyIncome', index='Education', columns='Attrition', aggfunc='mean')
    pivot_income.to_excel(writer, sheet_name='Charts', startrow=20, startcol=0)

    chart2 = workbook.add_chart({'type': 'column'})
    chart2.add_series({
        'name':       'Attrition = Yes',
        'categories': ['Charts', 21, 0, 21 + len(pivot_income)-1, 0],
        'values':     ['Charts', 21, 2, 21 + len(pivot_income)-1, 2],
    })
    chart2.set_title({'name': 'Avg Monthly Income by Education & Attrition'})
    worksheet.insert_chart('E20', chart2)

    # Detailed data sheet
    df.to_excel(writer, sheet_name='Detailed_Data', index=False)



# Implementing the "smart rule-based risk engine"
def risk_assessment(row):
    risk_points = 0
    if row['OverTime'] == 'Yes':
        risk_points += 1
    if row['JobSatisfaction'] == 'Low':
        risk_points += 1
    if row['YearsSinceLastPromotion'] >= 4:
        risk_points += 1
    if risk_points >= 3:
        return 'High'
    elif risk_points == 2:
        return 'Medium'
    else:
        return 'Low'

# Apply risk assessment
df['Attrition_Risk'] = df.apply(risk_assessment, axis=1)

# Prepare Risk Distribution for chart
risk_distribution = df['Attrition_Risk'].value_counts().reset_index()
risk_distribution.columns = ['Risk_Level', 'Count']

# Prepare Attrition by Department
attrition_dept = df[df['Attrition'] == 'Yes'].groupby('Department').size().reset_index(name='Attrition_Count')

# Prepare Attrition Rate by WorkLifeBalance
attrition_wlb = df[df['Attrition'] == 'Yes'].groupby('WorkLifeBalance').size().reset_index(name='Attrition_Count')

# Prepare Overtime Impact on Attrition
overtime_attrition = df.groupby(['OverTime', 'Attrition']).size().unstack().fillna(0)

# Generate the enhanced Excel Dashboard
enhanced_output = "../ds/Employee_Attrition_Advanced_Dashboard.xlsx"

with pd.ExcelWriter(enhanced_output, engine='xlsxwriter') as writer:
    # Cover Sheet
    cover_df = pd.DataFrame({
        'Employee Attrition Dashboard': [''],
        'Generated On': [pd.Timestamp.now().strftime('%Y-%m-%d')],
        'Total Employees': [total_employees],
        'Attrition Rate (%)': [attrition_rate],
        'High Risk Employees': [(df['Attrition_Risk'] == 'High').sum()]
    })
    cover_df.to_excel(writer, sheet_name='Dashboard', index=False)

    workbook  = writer.book
    dashboard_ws = writer.sheets['Dashboard']
    header_format = workbook.add_format({'bold': True, 'font_size':14})
    dashboard_ws.set_row(0, None, header_format)

    # Summary Sheet
    summary_df.to_excel(writer, sheet_name='Summary', index=False)

    # Charts Sheet
    risk_distribution.to_excel(writer, sheet_name='Charts', startrow=1, startcol=0)
    worksheet = writer.sheets['Charts']

    # Risk Distribution Chart
    chart_risk = workbook.add_chart({'type': 'pie'})
    chart_risk.add_series({
        'name': 'Attrition Risk Distribution',
        'categories': ['Charts', 2, 0, 2 + len(risk_distribution)-1, 0],
        'values':     ['Charts', 2, 1, 2 + len(risk_distribution)-1, 1],
    })
    chart_risk.set_title({'name': 'Attrition Risk Levels'})
    worksheet.insert_chart('E2', chart_risk)

    # Attrition by Department Chart
    attrition_dept.to_excel(writer, sheet_name='Charts', startrow=20, startcol=0)
    chart_dept = workbook.add_chart({'type': 'column'})
    chart_dept.add_series({
        'name': 'Attrition Count',
        'categories': ['Charts', 21, 0, 21 + len(attrition_dept)-1, 0],
        'values':     ['Charts', 21, 1, 21 + len(attrition_dept)-1, 1],
    })
    chart_dept.set_title({'name': 'Attrition by Department'})
    worksheet.insert_chart('E20', chart_dept)

    # Attrition by WorkLife Balance
    attrition_wlb.to_excel(writer, sheet_name='Charts', startrow=40, startcol=0)
    chart_wlb = workbook.add_chart({'type': 'column'})
    chart_wlb.add_series({
        'name': 'Attrition Count',
        'categories': ['Charts', 41, 0, 41 + len(attrition_wlb)-1, 0],
        'values':     ['Charts', 41, 1, 41 + len(attrition_wlb)-1, 1],
    })
    chart_wlb.set_title({'name': 'Attrition by WorkLife Balance'})
    worksheet.insert_chart('E40', chart_wlb)

    # Overtime Impact on Attrition
    overtime_attrition.to_excel(writer, sheet_name='Charts', startrow=60, startcol=0)
    chart_ot = workbook.add_chart({'type': 'column'})
    chart_ot.add_series({
        'name': 'No Attrition',
        'categories': ['Charts', 61, 0, 61 + len(overtime_attrition)-1, 0],
        'values':     ['Charts', 61, 1, 61 + len(overtime_attrition)-1, 1],
    })
    chart_ot.add_series({
        'name': 'Attrition',
        'categories': ['Charts', 61, 0, 61 + len(overtime_attrition)-1, 0],
        'values':     ['Charts', 61, 2, 61 + len(overtime_attrition)-1, 2],
    })
    chart_ot.set_title({'name': 'Overtime Impact on Attrition'})
    worksheet.insert_chart('E60', chart_ot)

    # Detailed Data with conditional formatting
    df.to_excel(writer, sheet_name='Detailed_Data', index=False)
    detail_ws = writer.sheets['Detailed_Data']

    # Apply conditional formatting for Attrition_Risk
    risk_col_idx = df.columns.get_loc("Attrition_Risk")
    detail_ws.conditional_format(1, risk_col_idx, len(df), risk_col_idx, {
        'type':     'text',
        'criteria': 'containing',
        'value':    'High',
        'format':   workbook.add_format({'bg_color': '#FF6666'})
    })
    detail_ws.conditional_format(1, risk_col_idx, len(df), risk_col_idx, {
        'type':     'text',
        'criteria': 'containing',
        'value':    'Medium',
        'format':   workbook.add_format({'bg_color': '#FFEB99'})
    })
